# Conditional Deep Convolutional Generative Adversarial Networks

## Imports

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
import time
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torchvision.utils import save_image

%matplotlib notebook
%config InlineBackend.figure_format = 'svg'

## Constants

In [ ]:
DATA_DIR = "/home/SharedData/intern_sayan/GAN/MNIST"

CHECKPOINT_DIR = "/home/SharedData/intern_sayan/GAN/"
CHECKPOINT_G = "mnist_cdcgan_gen.pth"
CHECKPOINT_D = "mnist_cdcgan_dis.pth"

SHOULD_CHECKPOINT = True
LOAD_CHECKPOINT = False

!mkdir -p cdcgan_images

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
CUDA = 0
BATCH_SIZE = 64
EPOCHS = 30

img_size = 32
n_channels = 1
img_shape = (n_channels, img_size, img_size)

lr_g = 0.002
lr_d = 0.002

latent_dim = 100
n_classes = 10

## Data

In [ ]:
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(DATA_DIR, train=True, download=True,
                   transform=transforms.Compose([
                       transforms.Resize(img_size),
                       transforms.ToTensor(),
                       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)


print(f"# train batches: {len(dataloader)}")

print(f"train image: {dataloader.dataset[0][0].size()}")

## Generator

In [ ]:
class Generator(nn.Module):
    def __init__(self, latent_dim, img_size, channels):
        super(Generator, self).__init__()
        
        self.init_size = img_size // 4
        
        self.label_emb = nn.Embedding(n_classes, n_classes)
        
        self.linear = nn.Linear(latent_dim + n_classes, 128 * (self.init_size ** 2))
    
        self.conv_net = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, channels, 3, stride=1, padding=1),
            nn.Tanh()
        )
        

    def forward(self, z, labels):
        y = self.label_emb(labels)
        x = torch.cat([y, z], dim=1)
        
        x = self.linear(x)
        x = x.view(x.size(0), 128, self.init_size, self.init_size)
        img = self.conv_net(x)
        
        img = img.view(img.size(0), *img_shape)
        
        return img

## Discriminator

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, channels, img_size):
        super(Discriminator, self).__init__()
        
        self.label_emb = nn.Embedding(n_classes, n_classes)

        pass

    def forward(self, img, labels):
        pass

## Training

In [ ]:
# Loss functions
adversarial_loss = torch.nn.BCELoss()

# Generator & Discriminator
generator = Generator(img_size=img_size,
                      latent_dim=latent_dim,
                      channels=n_channels)
discriminator = Discriminator(img_size=img_size,
                              channels=n_channels)

def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm2d') != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

if CUDA is not None:
    adversarial_loss.cuda()
    generator.cuda()
    discriminator.cuda()
    
if not LOAD_CHECKPOINT:
    # Init weights
    generator.apply(weights_init_normal)
    discriminator.apply(weights_init_normal)
    
    
if LOAD_CHECKPOINT:
    generator.load_state_dict(torch.load(os.path.join(CHECKPOINT_DIR,
                                                      CHECKPOINT_G)))
    discriminator.load_state_dict(torch.load(os.path.join(CHECKPOINT_DIR,
                                                          CHECKPOINT_D)))
    
# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr_g, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr_d, betas=(0.5, 0.999))


# Utils
FloatTensor = torch.cuda.FloatTensor if CUDA is not None else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if CUDA is not None else torch.LongTensor   

In [ ]:
from IPython.display import clear_output
from tqdm import tqdm_notebook

losses = {"generator" : list(), "discriminator": list()}

def display_progress(delta, epoch, epoch_d_loss, epoch_g_loss):
    clear_output(True)
    
    print(f"[Epoch {(epoch+1):3}/{EPOCHS}] [D loss: {epoch_d_loss:.8f}] [G loss: {epoch_g_loss:.8f}] in {delta:.2f} secs ")
    
    plt.plot(range(len(losses["generator"])), losses["generator"], "tab:blue")
    plt.plot(range(len(losses["discriminator"])), losses["discriminator"], "tab:orange")

    plt.legend(["generator", "discriminator"])
    
    plt.show()

    
def sample_image(epoch):
    """Saves a grid of generated digits ranging from 0 to n_classes"""
    n_row = n_classes
    # Sample noise
    z = torch.autograd.Variable(FloatTensor(np.random.normal(0, 1, (n_row**2, latent_dim))), requires_grad=False)
    
    labels = np.array([np.random.randn(latent_dim) for _ in range(n_row)])
    labels = torch.autograd.Variable(FloatTensor(labels), requires_grad=False)

    gen_imgs = generator(z, labels)

    save_image(gen_imgs.data, "dcgan_images/epoch_{}.png".format(epoch), nrow=n_row, normalize=True)


for epoch in range(EPOCHS):
    epoch_g_loss, epoch_d_loss = 0,0
    delta = time.time()
    
    for batch_idx, (imgs, labels) in tqdm_notebook(enumerate(dataloader)):
        batch_size = imgs.shape[0]

        # Adversarial ground truths
        valid = torch.autograd.Variable(FloatTensor(batch_size, 1).fill_(1.0), requires_grad=False)
        fake = torch.autograd.Variable(FloatTensor(batch_size, 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = torch.autograd.Variable(imgs.type(FloatTensor))        
        
        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise and labels as generator input
        z = torch.autograd.Variable(FloatTensor(np.random.normal(0, 1, (batch_size, latent_dim))))
        gen_labels = torch.autograd.Variable(LongTensor(np.random.randint(0, n_classes, batch_size)))

        # Generate a batch of images
        gen_imgs = generator(z, gen_labels)

        # Loss measures generator's ability to fool the discriminator
        validity = discriminator(gen_imgs, gen_labels)
        g_loss = adversarial_loss(validity, valid)

        g_loss.backward()
        optimizer_G.step()

        
        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Loss for real images
        validity_real = discriminator(real_imgs, labels)
        d_real_loss = adversarial_loss(validity_real, valid)

        # Loss for fake images
        validity_fake = discriminator(gen_imgs.detach(), gen_labels)
        d_fake_loss = adversarial_loss(validity_fake, fake)

        # Total discriminator loss
        d_loss = (d_real_loss + d_fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()
        
        
        epoch_g_loss += g_loss.detach().cpu().float()
        epoch_d_loss += d_loss.detach().cpu().float()        
        
    delta = time.time() - delta
    
    sample_image(epoch)
    
    losses["generator"].append(epoch_g_loss)
    losses["discriminator"].append(epoch_d_loss)
    
    if SHOULD_CHECKPOINT:
        torch.save(generator.state_dict(), os.path.join(CHECKPOINT_DIR,
                                                        CHECKPOINT_G))
        torch.save(discriminator.state_dict(), os.path.join(CHECKPOINT_DIR,
                                                            CHECKPOINT_D))
    
    display_progress(delta, epoch, epoch_d_loss, epoch_g_loss)
        

## Visualisation

In [ ]:
plt.plot(range(len(losses["discriminator"])), losses["discriminator"])

In [ ]:
plt.plot(range(len(losses["generator"])), losses["generator"])

In [ ]:
# Random sampling for inference

n_row = n_classes
n_col = 5


fig = plt.figure()
fig.set_figheight(24)
fig.set_figwidth(10)

for row in range(n_row):
    z = torch.autograd.Variable(FloatTensor(np.random.normal(0, 1, (n_col, latent_dim))))
    
    image = generator(z).detach().cpu().numpy()
        
    for col in range(n_col):
        index = row * n_col + col
        a = fig.add_subplot(n_row, n_col, index + 1)
        
        plt.imshow(image[col].reshape(img_size, img_size))